## Importing Packages

In [1]:
using CSV, DataFrames, PyCall, GLM, MLBase, Combinatorics

In [4]:
using Plots, StatsPlots, Plots.PlotMeasures
plotlyjs();

In [3]:
@pyimport statsmodels.stats.outliers_influence as vif
@pyimport sklearn.model_selection as sk
@pyimport imblearn
# @pyimport statsmodels.api as sm

## The data

In [5]:
brca = CSV.read("brca_df.csv", DataFrame, normalizenames= true);
first(brca,5)

Row,Age,Gender,Protein1,Protein2,Protein3,Protein4,HER2_status,II,I,Infiltrating_Ductal_Carcinoma,Infiltrating_Lobular_Carcinoma,Modified_Radical_Mastectomy,Lumpectomy,Simple_Mastectomy,Patient_Status
,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,36,0,0.080353,0.42638,0.54715,0.27368,0,0,0,1,0,1,0,0,1
2,43,0,-0.42032,0.57807,0.61447,-0.031505,0,1,0,0,0,0,1,0,0
3,69,0,0.21398,1.3114,-0.32747,-0.23426,0,0,0,1,0,0,0,0,1
4,56,0,0.34509,-0.21147,-0.19304,0.12427,0,1,0,1,0,1,0,0,1
5,56,0,0.22155,1.9068,0.52045,-0.31199,0,1,0,1,0,0,0,0,0


In [6]:
describe(brca)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Age,58.7256,29,58.0,90,0,Int64
2,Gender,0.0126183,0,0.0,1,0,Int64
3,Protein1,-0.0272321,-2.1446,0.0056486,1.5936,0,Float64
4,Protein2,0.949557,-0.97873,0.99713,3.4022,0,Float64
5,Protein3,-0.095104,-1.6274,-0.19304,2.1934,0,Float64
6,Protein4,0.00671301,-2.0255,0.038522,1.6299,0,Float64
7,HER2_status,0.0914826,0,0.0,1,0,Int64
8,II,0.567823,0,1.0,1,0,Int64
9,I,0.189274,0,0.0,1,0,Int64


## Basic Logistic Regression

In [7]:
fm = @formula(Patient_Status ~ Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + 
                                Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + 
                                Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy);
logit = glm(fm, brca, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Patient_Status ~ 1 + Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────────────
                                     Coef.  Std. Error      z  Pr(>|z|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                      0.238205   0.60928      0.39    0.6958  -0.955963   1.43237
Age                              0.0015643  0.00670538   0.23    0.8155  -0.011578   0.0147066
Gender                          -0.316648   0.7

In [8]:
prediction = GLM.predict(logit,brca);
prediction = [if x < 0.5 0 else 1 end for x in prediction]
confmat = MLBase.roc(brca.Patient_Status, prediction);
[:ConfMat :PredictedAlive :PredictedDead; :ActualAlive confmat.tp confmat.fn; :ActualDead confmat.fp confmat.tn]

3×3 Matrix{Any}:
 :ConfMat         :PredictedAlive   :PredictedDead
 :ActualAlive  255                 0
 :ActualDead    62                 0

In [9]:
println("Overall Accuracy: ",round((confmat.tp + confmat.tn)*100/length(prediction), digits = 2),"%")
println("AIC= ",MLBase.aic(logit))
println("BIC= ",MLBase.bic(logit))
println("Recall= ",MLBase.recall(confmat))
println("Specificity= ",MLBase.true_negative_rate(confmat))
println("Fall-out= ",MLBase.false_positive_rate(confmat))
println("Precision= ",MLBase.precision(confmat))

Overall Accuracy: 80.44%
AIC= 331.6236426365257
BIC= 388.00716924468486
Recall= 1.0
Specificity= 0.0
Fall-out= 1.0
Precision= 0.804416403785489


In [10]:
recall_1 = []
fallout_1 = []
for t in range(start=0,step=0.005, stop=1)
    r_val = []
    f_val = []
    for i in 1:100
    fm = @formula(Patient_Status ~ Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + 
                                Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + 
                                Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy);
    logit = glm(fm, brca, Binomial(), ProbitLink())

    prediction = GLM.predict(logit,brca);
    prediction = [if x < t 0 else 1 end for x in prediction]
    confmat = MLBase.roc(brca.Patient_Status, prediction);
    push!(r_val,MLBase.recall(confmat))
    push!(f_val,MLBase.false_positive_rate(confmat))
    end
push!(recall_1,mean(r_val))
push!(fallout_1,mean(f_val))
end

In [11]:
plot(fallout_1,recall_1, legendposition=:outertopright, label = "ROC curve", lw = 2)
plot!(0:0.005:1,0:0.005:1, label = "Random guess", size = (800,400), lw = 1.5, xlabel = "False Positive Rate",
        ylabel = "True Positive Rate")

In [12]:
vif_data = DataFrame(Predictors = names(brca),
    VIF = [vif.variance_inflation_factor(Array(brca), i) for i in 0: size(brca)[2]-1])

Row,Predictors,VIF
,String,Float64
1,Age,16.0918
2,Gender,1.06151
3,Protein1,1.22344
4,Protein2,2.66677
5,Protein3,1.27453
6,Protein4,1.12297
7,HER2_status,1.15197
8,II,3.4601
9,I,1.99962


In [13]:
l = []
for j in unique(brca.Patient_Status)
    push!(l,length([brca.Patient_Status[i] for i in 1:length(brca.Patient_Status) if brca.Patient_Status[i]==j])/length(brca.Patient_Status))
end
l = [Float64(x) for x in l];
bar(["Alive", "Dead"], l, ylim = (0,1), framestyle = :box, legend = false, size = (400,450), xlabel = "Patient Status")

## Class Imbalance

In [14]:
smote = imblearn.over_sampling.SMOTE();

In [15]:
X = Array(brca[:,1:14]);
y = brca[:,:Patient_Status];

In [16]:
X, y = smote.fit_resample(X, y);

In [17]:
brca = DataFrame(X, ["Age", "Gender", "Protein1", "Protein2", "Protein3", "Protein4", "HER2_status", "II", "I", 
                    "Infiltrating_Ductal_Carcinoma", "Infiltrating_Lobular_Carcinoma", "Modified_Radical_Mastectomy", 
                    "Lumpectomy", "Simple_Mastectomy"]);
insertcols!(brca, 15, :Patient_Status => y);

In [18]:
size(brca)

(510, 15)

In [19]:
describe(brca)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Age,58.3096,29.0,57.0628,90.0,0,Float64
2,Gender,0.0207267,0.0,0.0,1.0,0,Float64
3,Protein1,-0.0201519,-2.1446,0.0453621,1.5936,0,Float64
4,Protein2,1.03445,-0.97873,1.0942,3.4022,0,Float64
5,Protein3,-0.108083,-1.6274,-0.177761,2.1934,0,Float64
6,Protein4,0.0278403,-2.0255,0.0617753,1.6299,0,Float64
7,HER2_status,0.0865358,0.0,0.0,1.0,0,Float64
8,II,0.565669,0.0,0.965714,1.0,0,Float64
9,I,0.171644,0.0,0.0,1.0,0,Float64


In [20]:
l = []
for j in unique(brca.Patient_Status)
    push!(l,length([brca.Patient_Status[i] for i in 1:length(brca.Patient_Status) if brca.Patient_Status[i]==j])/length(brca.Patient_Status))
end
l = [Float64(x) for x in l];
bar(["Alive", "Dead"], l, ylim = (0,1), framestyle = :box, legend = false, size = (400,450), xlabel = "Patient Status")

## SMOTE Logistic Regression

In [21]:
fm = @formula(Patient_Status ~ Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + 
                                Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + 
                                Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy);
logit = glm(fm, brca, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Patient_Status ~ 1 + Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────────────────
                                      Coef.  Std. Error      z  Pr(>|z|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                     -0.796686    0.464057    -1.72    0.0860  -1.70622      0.112849
Age                              0.00367485  0.00468102   0.79    0.4324  -0.00549978   0.0128495
Gender                         

In [22]:
prediction = GLM.predict(logit,brca);
prediction = [if x < 0.5 0 else 1 end for x in prediction]
confmat = MLBase.roc(brca.Patient_Status, prediction);
[:ConfMat :PredictedAlive :PredictedDead; :ActualAlive confmat.tp confmat.fn; :ActualDead confmat.fp confmat.tn]

3×3 Matrix{Any}:
 :ConfMat         :PredictedAlive     :PredictedDead
 :ActualAlive  152                 103
 :ActualDead    98                 157

In [23]:
println("Overall Accuracy: ",round((confmat.tp + confmat.tn)*100/length(prediction), digits = 2),"%")
println("AIC= ",MLBase.aic(logit))
println("BIC= ",MLBase.bic(logit))
println("Recall= ",MLBase.recall(confmat))
println("Specificity= ",MLBase.true_negative_rate(confmat))
println("Fall-out= ",MLBase.false_positive_rate(confmat))
println("Precision= ",MLBase.precision(confmat))

Overall Accuracy: 60.59%
AIC= 697.6601853717522
BIC= 761.1763462575277
Recall= 0.596078431372549
Specificity= 0.615686274509804
Fall-out= 0.3843137254901961
Precision= 0.608


In [24]:
recall_2 = []
fallout_2 = []
for t in range(start=0,step=0.005, stop=1)
    r_val = []
    f_val = []
    for i in 1:100
    fm = @formula(Patient_Status ~ Age + Gender + Protein1 + Protein2 + Protein3 + Protein4 + HER2_status + I + II + 
                                Infiltrating_Lobular_Carcinoma + Infiltrating_Ductal_Carcinoma + 
                                Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy);
    logit = glm(fm, brca, Binomial(), ProbitLink())

    prediction = GLM.predict(logit,brca);
    prediction = [if x < t 0 else 1 end for x in prediction]
    confmat = MLBase.roc(brca.Patient_Status, prediction);
    push!(r_val,MLBase.recall(confmat))
    push!(f_val,MLBase.false_positive_rate(confmat))
    end
push!(recall_2,mean(r_val))
push!(fallout_2,mean(f_val))
end

In [25]:
plot(fallout_1,recall_1, legendposition=:outertopright, label = "ROC curve 1", lw = 2)
plot!(fallout_2,recall_2, legendposition=:outertopright, label = "ROC curve 2", lw = 2)
plot!(0:0.005:1,0:0.005:1, label = "Random guess", size = (800,400), lw = 1.5, xlabel = "False Positive Rate",
        ylabel = "True Positive Rate")

In [26]:
vif_data = DataFrame(Predictors = names(brca),
    VIF = [vif.variance_inflation_factor(Array(brca), i) for i in 0: size(brca)[2]-1])

Row,Predictors,VIF
,String,Float64
1,Age,16.7191
2,Gender,1.1397
3,Protein1,1.27132
4,Protein2,3.30411
5,Protein3,1.24837
6,Protein4,1.1738
7,HER2_status,1.1649
8,II,3.61565
9,I,1.92867


## Best Subset Selection

In [27]:
function best_subset_selection(df)
    dev = []
    for num in 1:length(names(df))-1
        val = []
        for i in collect(combinations(1:length(names(df))-1,num))
            logreg = glm(Array(df[:,i]), Array(df[:,15]), Binomial(), ProbitLink())
            push!(val,MLBase.deviance(logreg))
        end
        push!(dev,collect(combinations(1:length(names(df))-1,num))[indexin(minimum(val),val)])
    end
    return [dev[i][1] for i in 1:length(names(df))-1]
end

best_subset_selection (generic function with 1 method)

In [28]:
dev = best_subset_selection(brca)

14-element Vector{Vector{Int64}}:
 [13]
 [4, 13]
 [4, 11, 13]
 [4, 11, 13, 14]
 [4, 11, 12, 13, 14]
 [2, 4, 11, 12, 13, 14]
 [2, 4, 6, 11, 12, 13, 14]
 [2, 3, 4, 6, 11, 12, 13, 14]
 [2, 3, 4, 6, 9, 11, 12, 13, 14]
 [2, 3, 4, 6, 8, 9, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [29]:
final = []
for d in dev
    logreg = glm(Array(brca[:,d]), Array(brca[:,15]), Binomial(), ProbitLink())
    push!(final,MLBase.aic(logreg))
end
println("Indices of the 'best' model are ",dev[indexin(minimum(final),final)[1]])
println("The predictors to the corresponding indices are\n",names(brca[:,dev[indexin(minimum(final),final)[1]]]))

Indices of the 'best' model are [2, 4, 6, 11, 12, 13, 14]
The predictors to the corresponding indices are
["Gender", "Protein2", "Protein4", "Infiltrating_Lobular_Carcinoma", "Modified_Radical_Mastectomy", "Lumpectomy", "Simple_Mastectomy"]


## Forward Step-wise Selection

In [29]:
function forward_stepwise_selection(df)
    dev = []
    comb = collect(combinations(1:length(names(df))-1,1))
    for num in 1:length(names(df))-1
        val = []
        for j in comb
            logreg = glm(Array(df[:,i]), Array(df[:,15]), Binomial(), ProbitLink())
            push!(val,MLBase.deviance(logreg))
        end
        push!(dev,comb[indexin(minimum(val),val)])
        comb = collect(combinations(1:length(names(df))-1,num+1)) 
        l = []
        for j in 1:length(comb)
            a = true
            for i in dev[end][1]
                a = a & (i in comb[j])
            end
        push!(l,a)
        end
        comb = comb[[i for i in l]]
    end
    return [dev[i][1] for i in 1:length(names(df))-1]
end

forward_stepwise_selection (generic function with 1 method)

In [64]:
dev = forward_stepwise_selection(brca)

14-element Vector{Vector{Int64}}:
 [13]
 [4, 13]
 [4, 9, 13]
 [4, 7, 9, 13]
 [4, 6, 7, 9, 13]
 [4, 6, 7, 9, 12, 13]
 [3, 4, 6, 7, 9, 12, 13]
 [3, 4, 6, 7, 9, 11, 12, 13]
 [2, 3, 4, 6, 7, 9, 11, 12, 13]
 [2, 3, 4, 6, 7, 9, 11, 12, 13, 14]
 [2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [31]:
final = []
for d in dev
    logreg = glm(Array(brca[:,d]), Array(brca[:,15]), Binomial(), ProbitLink())
    push!(final,MLBase.aic(logreg))
end
println("Indices of the 'best' model are ",dev[indexin(minimum(final),final)[1]])
println("The predictors to the corresponding indices are\n",names(brca[:,dev[indexin(minimum(final),final)[1]]]))

Indices of the best model are [3, 4, 6, 7, 9, 12, 13, 14]
The best predictors for the corresponding indices are 
["Protein1", "Protein2", "Protein4", "HER2_status", "I", "Modified_Radical_Mastectomy", "Lumpectomy", "Simple_Mastectomy"]
AIC for the best model is 673.7690988832999


## Backward Step-wise Selection

In [65]:
function backward_stepwise_selection(df)
    dev = []
    index = Set([s for s in 1:length(names(df))-1])
    comb = collect(combinations(sort(collect(index)),length(names(df))-1))        
    for num in length(names(df))-1:-1:1
        val = []
        for j in comb
            logreg = sm.Logit(Array(brca[:,[15]]), Array(brca[:,j])).fit(disp=0);
            push!(val,-2*logreg.llf)
        end
        push!(dev,comb[indexin(minimum(val),val)])
        comb = collect(combinations(sort(collect(index)),num-1))
    end  
    return [dev[i][1] for i in 1:length(names(df))-1]
end

backward_stepwise_selection (generic function with 1 method)

In [66]:
dev = backward_stepwise_selection(brca)

14-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]
 [2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14]
 [3, 4, 6, 7, 8, 9, 10, 12, 13, 14]
 [3, 4, 6, 7, 9, 10, 12, 13, 14]
 [4, 6, 7, 9, 10, 12, 13, 14]
 [3, 4, 6, 7, 9, 12, 13]
 [4, 6, 7, 9, 12, 13]
 [4, 6, 7, 9, 13]
 [4, 7, 9, 13]
 [4, 9, 13]
 [4, 13]
 [13]

In [34]:
final = []
for d in dev
    logreg = sm.Logit(Array(brca[:,[15]]),Array(brca[:,d])).fit(disp=0);
    push!(final,logreg.aic)
end
println("Indices of the best model are ",dev[indexin(minimum(final),final)[1]])
println("The best predictors for the corresponding indices are \n",names(brca[:,dev[indexin(minimum(final),final)[1]]]))
println("AIC for the best model is ",minimum(final))

Indices of the best model are [3, 4, 6, 7, 9, 12, 13, 14]
The best predictors for the corresponding indices are 
["Protein1", "Protein2", "Protein4", "HER2_status", "I", "Modified_Radical_Mastectomy", "Lumpectomy", "Simple_Mastectomy"]
AIC for the best model is 673.7690988832999


## Further Assessment

In [27]:
names(brca[:,[3,4,6,9,11,12,13,14]])

8-element Vector{String}:
 "Protein1"
 "Protein2"
 "Protein4"
 "I"
 "Infiltrating_Lobular_Carcinoma"
 "Modified_Radical_Mastectomy"
 "Lumpectomy"
 "Simple_Mastectomy"

In [37]:
# ModelSelection StatsModels
logreg1 = sm.Logit(Array(brca[:,[15]]), Array(brca[:,[3,4,6,9,11,12,13,14]])).fit(disp=0);
println("aic= ",logreg1.aic)
println("bic= ",logreg1.bic)
logreg1.summary()

aic= 680.6606112493263
bic= 714.5358970550733


PyObject <class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  510
Model:                          Logit   Df Residuals:                      502
Method:                           MLE   Df Model:                            7
Date:                Wed, 28 Dec 2022   Pseudo R-squ.:                 0.05990
Time:                        20:25:00   Log-Likelihood:                -332.33
converged:                       True   LL-Null:                       -353.51
Covariance Type:            nonrobust   LLR p-value:                 4.453e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2984      0.198      1.507      0.132      -0.090       0.686
x2            -0.4114      0.094     -4.361      0.000      -0.596      -0.227
x3            -0.4274      0.167     -2.563      0.010      -0.754      -0.101
x4             0.6933      0.279      2.487      0.013       0.147       1.240
x5             0.3372      0.218      1.547      0.122      -0.090       0.764
x6             0.3577      0.210      1.706      0.088      -0.053       0.769
x7             1.0437      0.280      3.726      0.000       0.495       1.593
x8             0.1706      0.231      0.740      0.459      -0.281       0.623
==============================================================================
"""

In [38]:
# ModelSelection GLM
fm = @formula(Patient_Status ~ Protein1+Protein2+Protein4+I+Infiltrating_Lobular_Carcinoma+Modified_Radical_Mastectomy+Lumpectomy+Simple_Mastectomy);
logit1 = glm(fm, brca, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Patient_Status ~ 1 + Protein1 + Protein2 + Protein4 + I + Infiltrating_Lobular_Carcinoma + Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────────────
                                    Coef.  Std. Error      z  Pr(>|z|)    Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                     -0.150895   0.14312    -1.05    0.2917  -0.431405     0.129615
Protein1                         0.147461   0.127821    1.15    0.2486  -0.103063     0.397985
Protein2                        -0.210223   0.0717739  -2.93    0.0034  -0.350897    -0.0695485
Protein4               

In [39]:
println("aic=", MLBase.aic(logit1))
println("bic=", MLBase.bic(logit1))

aic=681.4438889141327
bic=719.5535854455981


In [40]:
# VIF ModelSelection
vif_data = DataFrame(Predictors =
    ["Protein1","Protein2","Protein4","I","Infiltrating_Lobular_Carcinoma",
    "Modified_Radical_Mastectomy","Simple_Mastectomy","Lumpectomy"],
    VIF = [vif.variance_inflation_factor(Array(brca[:,[3,4,6,9,11,12,13,14]]), i)
        for i in 0: size(brca[:,[3,4,6,9,11,12,13,14]])[2]-1])

Row,Predictors,VIF
,String,Float64
1,Protein1,1.10297
2,Protein2,1.72644
3,Protein4,1.08618
4,I,1.2271
5,Infiltrating_Lobular_Carcinoma,1.26632
6,Modified_Radical_Mastectomy,1.27859
7,Simple_Mastectomy,1.28183
8,Lumpectomy,1.18817


## ROC

In [85]:
# ModelSelection GLM
fm = @formula(Patient_Status ~ Protein1+Protein2+Protein4+I+Infiltrating_Lobular_Carcinoma+Modified_Radical_Mastectomy+Lumpectomy+Simple_Mastectomy);
logit1 = glm(fm, brca, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Patient_Status ~ 1 + Protein1 + Protein2 + Protein4 + I + Infiltrating_Lobular_Carcinoma + Modified_Radical_Mastectomy + Lumpectomy + Simple_Mastectomy

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────────────
                                     Coef.  Std. Error      z  Pr(>|z|)   Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                     -0.0896651   0.141451   -0.63    0.5262  -0.366904    0.187574
Protein1                         0.293236    0.123585    2.37    0.0177   0.0510127   0.535458
Protein2                        -0.164802    0.0718117  -2.29    0.0217  -0.30555    -0.0240536
Protein4               

In [86]:
prediction = GLM.predict(logit,brca);
prediction = [if x < 0.5 0 else 1 end for x in prediction]
confmat = MLBase.roc(brca.Patient_Status, prediction);
[:ConfMat :PredictedAlive :PredictedDead; :ActualAlive confmat.tp confmat.fn; :ActualDead confmat.fp confmat.tn]

3×3 Matrix{Any}:
 :ConfMat         :PredictedAlive     :PredictedDead
 :ActualAlive  148                 107
 :ActualDead   103                 152

In [87]:
println("Overall Accuracy: ",round((confmat.tp + confmat.tn)*100/length(prediction), digits = 2),"%")
println("AIC= ",MLBase.aic(logit))
println("BIC= ",MLBase.bic(logit))
println("Recall= ",MLBase.recall(confmat))
println("Specificity= ",MLBase.true_negative_rate(confmat))
println("Fall-out= ",MLBase.false_positive_rate(confmat))
println("Precision= ",MLBase.precision(confmat))

Overall Accuracy: 58.82%
AIC= 699.760786520301
BIC= 733.6360723260481
Recall= 0.5803921568627451
Specificity= 0.596078431372549
Fall-out= 0.403921568627451
Precision= 0.5896414342629482


In [100]:
recall_3 = []
fallout_3 = []
for t in range(start=0,step=0.005, stop=1)
    r_val = []
    f_val = []
    for i in 1:100
    fm = @formula(Patient_Status ~ Protein1+Protein2+Protein4+I+Infiltrating_Lobular_Carcinoma+Modified_Radical_Mastectomy+Lumpectomy+Simple_Mastectomy);
    logit = glm(fm, brca, Binomial(), ProbitLink())

    prediction = GLM.predict(logit,brca);
    prediction = [if x < t 0 else 1 end for x in prediction]
    confmat = MLBase.roc(brca.Patient_Status, prediction);
    push!(r_val,MLBase.recall(confmat))
    push!(f_val,MLBase.false_positive_rate(confmat))
    end
push!(recall_3,mean(r_val))
push!(fallout_3,mean(f_val))
end

In [101]:
plot(fallout_1,recall_1, legendposition=:outertopright, label = "ROC curve 1", lw = 2, c="yellow3")
plot!(fallout_2,recall_2, legendposition=:outertopright, label = "ROC curve 2", lw = 2, c="red")
plot!(fallout_3,recall_3, legendposition=:outertopright, label = "ROC curve 3", lw = 2, c="green3")
plot!(0:0.005:1,0:0.005:1, label = "Random guess", size = (800,400), lw = 1.5, xlabel = "False Positive Rate",
        ylabel = "True Positive Rate", c="black")

## AUC Calculation

In [102]:
function auc(fallout,recall)
    integral = 0
    f = [i for i in keys(sort(Dict([(fallout[i],recall[i]) for i in 1:length(fallout)])))]
    r = [i for i in values(sort(Dict([(fallout[i],recall[i]) for i in 1:length(fallout)])))];
    for i in 1:length(f)-1
        integral = integral + 0.5*(r[i+1]+r[i])*(f[i+1]-f[i])
    end
    return integral
end

auc (generic function with 1 method)

In [103]:
auc(fallout_1,recall_1)

0.6322264389626815

In [104]:
auc(fallout_2,recall_2)

0.6506574394463668

In [105]:
auc(fallout_3,recall_3)

0.6150403690888118

## Validation Set Approach

In [237]:
test_acc = []
train_acc=  []
sens = []
spec = []
for i in 1:1000
    X_train, X_test, y_train, y_test = sk.train_test_split(Array(brca[:,[2, 3, 4, 6, 9, 11, 13]]),
        brca[:,:Patient_Status], test_size = 0.3);
    
    brca_train = DataFrame(Gender = X_train[:,1], Protein1 = X_train[:,:2], Protein2 = X_train[:,:3], Protein4 = X_train[:,:4], 
        I = X_train[:,:5], Infiltrating_Lobular_Carcinoma = X_train[:,:6], Lumpectomy = X_train[:,:7], Patient_Status = y_train);
    
    brca_test = DataFrame(Gender = X_test[:,1], Protein1 = X_test[:,:2], Protein2 = X_test[:,:3], Protein4 = X_test[:,:4], 
        I = X_test[:,:5], Infiltrating_Lobular_Carcinoma = X_test[:,:6],Lumpectomy = X_test[:,:7], Patient_Status = y_test);

    fm = @formula(Patient_Status ~ Gender+Protein1+Protein2+Protein4+I+Infiltrating_Lobular_Carcinoma+Lumpectomy);
    logit = glm(fm, brca_train, Binomial(), ProbitLink());
    prediction = [if x < 0.5 0 else 1 end for x in GLM.predict(logit,brca_test)];
    confmat = MLBase.roc(brca_test.Patient_Status, prediction);
    push!(test_acc, (confmat.tp + confmat.tn)/size(brca_test)[1])
    push!(sens, (confmat.tp/confmat.tp + confmat.fn))
    push!(spec, (confmat.tn/confmat.tn + confmat.fp))
    
    prediction_train = [if x < 0.5 0 else 1 end for x in GLM.predict(logit,brca_train)];
    confmat_train = MLBase.roc(brca_train.Patient_Status, prediction_train);
    push!(train_acc, (confmat_train.tp + confmat_train.tn)/size(brca_train)[1])
end

In [238]:
println(median(test_acc))
boxplot([test_acc])

0.5686274509803921


## Correlation Matrix

In [70]:
heatmap([abs(i) for i in cor(Array(brca[:,[2,3,4,6,9,11,13]]))], colorscale = :Greys, reversescale = true,
        grid = false, xticks=(1:7,names(brca[:,[2,3,4,6,9,11,13]])), yticks=(1:7,names(brca[:,[2,3,4,6,9,11,13]])),
        xrotation=30, xtickfontsize=8, ytickfontsize=8, size = (800,500))

## K-fold

In [164]:
function kfold(brca,k)
    train = collect(Kfold(size(brca)[1],k))
    test = []
    cv = []
    for i in 1:k
        push!(test,sort(collect(setdiff(Set(1:size(brca)[1]), train[i]))))
    end
    for i in 1:k
        X_train, X_test, y_train, y_test = Array(brca[train[i],[3,4,6,9,11,12,13,14]]), Array(brca[test[i],[3,4,6,9,11,12,13,14]]),
                                        Array(brca[train[i],15]), Array(brca[test[i],15]);

        brca_train = DataFrame(Protein1 = X_train[:,1], Protein2 = X_train[:,:2], Protein4 = X_train[:,:3], I = X_train[:,:4], 
            Infiltrating_Lobular_Carcinoma = X_train[:,:5], Modified_Radical_Mastectomy = X_train[:,:6],
            Lumpectomy = X_train[:,:7], Simple_Mastectomy = X_train[:,:8],Patient_Status = y_train);

        brca_test = DataFrame(Protein1 = X_test[:,1], Protein2 = X_test[:,:2], Protein4 = X_test[:,:3], I = X_test[:,:4], 
            Infiltrating_Lobular_Carcinoma = X_test[:,:5], Modified_Radical_Mastectomy = X_test[:,:6],
            Lumpectomy = X_test[:,:7], Simple_Mastectomy = X_test[:,:8],Patient_Status = y_test);

        fm = @formula(Patient_Status ~ Protein1+Protein2+Protein4+I+Infiltrating_Lobular_Carcinoma+Modified_Radical_Mastectomy+Lumpectomy+Simple_Mastectomy);
        logit = glm(fm, brca_train, Binomial(), ProbitLink());
        prediction = [if x < 0.5 0 else 1 end for x in GLM.predict(logit,brca_test)];
        confmat = MLBase.roc(brca_test.Patient_Status, prediction);
        push!(cv, (confmat.tp + confmat.tn)/size(brca_test)[1])
    end
    return mean(cv)
end

kfold (generic function with 1 method)

In [185]:
t = []
for i in 1:1000
    push!(t, kfold(brca,3))
end

In [188]:
mean(t)

0.5478156862745115